In [ ]:
import json

protein_info_file = "data\\protein_information.json"
training_file = "data\\f_covae_training.json"

In [ ]:
with open(protein_info_file, 'r') as file:
    proteins_raw = json.load(file)

proteins_prep = {}
for id in proteins_raw.keys():
    proteins_prep[id] = {}

GO Terms

In [ ]:
go_terms = {"mf": [], "cc": [], "bp": []}
for p_id, p_data in proteins_raw.items():
    for p_term in p_data["go_terms"]:
        match p_term[1][0]:
            case "F": go_terms["mf"].append(p_term[0])
            case "P": go_terms["bp"].append(p_term[0])
            case "C": go_terms["cc"].append(p_term[0])

for aspect in go_terms.keys():
    go_terms[aspect].sort()

In [ ]:
for id in proteins_prep.keys():
    proteins_prep[id]["go_terms"] = {}

for aspect, terms in go_terms.items():
    for p_id, p_data in proteins_raw.items():
        p_encoded_go = [0] * len(go_terms[aspect])
        for p_go in p_data["go_terms"]:
            if p_go[0] in terms:
                index = terms.index(p_go[0])
                p_encoded_go[index] = 1
        proteins_prep[p_id]["go_terms"][aspect] = p_encoded_go


In [ ]:
#test
test_protein_terms = proteins_raw["A0A0X8BZN2"]["go_terms"]
for term in test_protein_terms:
    if "F:" in term[1]: print(f"F:{term[0]}: {go_terms['mf'].index(term[0])}")
    elif "C:" in term[1]: print(f"C:{term[0]}: {go_terms['cc'].index(term[0])}")
    elif "P:" in term[1]: print(f"P:{term[0]}: {go_terms['bp'].index(term[0])}")

print()
for aspect in go_terms.keys():
    c = -1
    for num in proteins_prep["A0A0X8BZN2"]["go_terms"][aspect]:
        c+=1
        if num == 1: print(c)
    print()

Motifs (DO NOT USE) - All motifs collected, too many features. Unsure of how consensus patterns work.

In [ ]:
motifs = []
for p_id, p_data in proteins_raw.items():
    for motif in p_data["motifs"]:
        if motif not in motifs:
            motifs.append(motif)

motifs.sort()
num_motifs = len(motifs)

In [ ]:
for p_id, p_data in proteins_raw.items():
    p_encoded_motifs = [0] * num_motifs
    for p_motif in p_data["motifs"]:
        index = motifs.index(p_motif)
        p_encoded_motifs[index] = 1
    proteins_prep[p_id]["motifs"] = p_encoded_motifs

In [ ]:
#test
test_protein_motifs = proteins_raw["A0A0X8BZN2"]["motifs"]
for motif in test_protein_motifs:
    print(f"{motif}: {motifs.index(motif)}")
print()
c = -1
for num in proteins_prep["A0A0X8BZN2"]["motifs"]:
    c+=1
    if num == 1: print(c)

Simple Biopython Features

In [ ]:
biopyfeatures = ["length", "m_weight", "instab_index", "isoele_point", "gravy", "sec_sruct_frac", "ext_coeff"]
for p_id, p_data in proteins_raw.items():
    for feature in biopyfeatures:
        proteins_prep[p_id][feature] = p_data[feature]


In [ ]:
#test
for feature in biopyfeatures:
    print(proteins_raw["A0A0X8BZN2"][feature])

print()

for feature in biopyfeatures:
    print(proteins_prep["A0A0X8BZN2"][feature])

Amino Count

In [ ]:
for p_id, p_data in proteins_raw.items():
    acids = []
    for acid in p_data["amino_count"].values():
        acids.append(acid)
    proteins_prep[p_id]["amino_count"] = acids

In [ ]:
#test
for acid in proteins_raw["A0A0X8BZN2"]["amino_count"].values():
    print(acid)

print()

for acid in proteins_prep["A0A0X8BZN2"]["amino_count"]:
    print(acid)
